<a href="https://colab.research.google.com/github/NogginBops/DD2363_VT23/blob/main/Lab2/report_lab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2: Iterative Methods**
**Julius Häger**

# **Abstract**

In this lab I implement two iterative methods for linear systems of equations. Jacobian iteration and Gauss-Seidel iteration. I also implement Newton's method for non-linear scalar equations.

#**About the code**

In [ ]:
"""This program is lab report in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2023 Julius Häger (juliusha@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

In [ ]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np

from IPython.display import display, Math

#try:
#    from dolfin import *; from mshr import *
#except ImportError as e:
#    !apt-get install -y -qq software-properties-common 
#    !add-apt-repository -y ppa:fenics-packages/fenics
#    !apt-get update -qq
#    !apt install -y --no-install-recommends fenics
#    from dolfin import *; from mshr import *
    
#import dolfin.common.plotting as fenicsplot

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D


# **Introduction**

To solve big systems the computational cost of direct methods for solving these systems becomes large. Iterative methods has potential to be much faster in this regard as they don't need to compute the exact solution, they can stop after some tolerance is reached.

Iterative methods often have the advantage that they only need to compute matrix-vector products. This makes it easy to exploit knowledge about the system, e.g. if the matrix is sparse it's easy to implement a sparse matrix-vector product.

The simplest method for iteratively solving linear systems of equations is Richardson iteration where the next value $\mathbf{x}^{(k+1)}$ is given by $\mathbf{x}^{(k+1)} = (I - \alpha A)\mathbf{x}^{(k)} + \mathbf{b}$, where $\alpha > 0$ is an arbitrary parameter of the method.

The convergance of Richardson iteration is given by $|| I - \alpha A ||$ so to get fast convergence we want $|| I - \alpha A||$ to be close to a zero matrix. This doesn't always happen, but what we can do is predcondition the system $Ax = b$ with some other matrix $B$ to get a new system of equations with the same solution as the first solution $BAx = Bb$. This gives us the new iteration $\mathbf{x}^{(k+1)} = (I - \alpha BA)\mathbf{x}^{(k)} + \alpha Bb$. To find a good predondition matrix we want to minimize $|| I - \alpha BA ||$ by finding a good matrix $B$.

The first method implemented for this is jacobi iteration. In jacobi iteration we use $B = \mathrm{diag}(A)^{-1}=D{-1}$ and set $\alpha = 1$. This gives us the iteration $\mathbf{x}^{(k+1)} = \mathbf{x}^{(k)} + D^{-1}(\mathbf{b} - A\mathbf{x}^{(k) })$ where multiplication by $D^{-1}$ can be implemented as element-wise division by the diagonal of $D$.

The second method implemented is called Gauss-Seidel iteration. Here the predondition matrix is $B = \mathrm{tril}(A)^{-1} = L^{-1}$ and $\alpha = 1$ which gives the iteration $L\mathbf{x}^{(k+1)} = \mathbf{b} - (A - L)\mathbf{x}^{(k)}$, from which $\mathbf{x}^{(k+1)}$ can be solved for with forward-substitution.

For non-linear scalar equations, solutions can be found using Newtons method. This method utilizes the derivative of the continous function to calculate the intersection of the tangent with the x-axis, which is then used for the next iteration. A version of this called the secant method doesn't require an explicit derivative, and instead calculates a secant from two initial guesses and uses the intersection with the x-axis as one of it's two gueses.

# **Method**

## Assignment 1: Jacobi iteration for $Ax=b$

To implement jacobi iteration we can take two routes. The first route is to implement a richardson iteration procedure and then pass it jacobi iteration preconditioned matrices; this is implemented in the `jacobi_iteration(A, b, TOL=1e-16)` procedure.
The other route is to utilize the knowledge that multiplication by $D^{-1}$ is equivalent to division of the diagonal elements of $D$. This version is implemented as `jacobi_iteration_direct(A, b, TOL=1e-16)`.

In [ ]:
# Assignment 1: Jacobi iteration for Ax=b

def inverse_diag(A : np.ndarray):
  return np.diag(np.reciprocal(np.diag(A)));

def richardson_iteration(A : np.ndarray, b : np.ndarray, alpha, TOL = 1e-16):
  x = np.zeros_like(b)
  r = b.copy()
  #display(Math(rf"|| A || = {np.linalg.norm(A, ord=2)}"))
  iteration = 0
  while np.linalg.norm(r) / np.linalg.norm(b) > TOL:
    #print(f"x{iteration} = {x}", "r", r, "x", x, "tol", np.linalg.norm(r) / np.linalg.norm(b))
    r = b - (A @ x)
    x = x + alpha * r
    iteration += 1

  #print(f"iterations = {iteration}")
  return x

def jacobi_iteration(A : np.ndarray, b : np.ndarray, TOL = 1e-16):
  Di = inverse_diag(A)
  M = Di @ A
  c = Di @ b
  #display(Math(rf"|| I - D^{{-1}}A || = {np.linalg.norm(M, ord=2)}"))
  return richardson_iteration(M, c, 1, TOL)

def jacobi_iteration_direct(A : np.ndarray, b : np.ndarray, TOL = 1e-16):
  x = np.zeros(b.size)
  r = b.copy()
  D = np.diag(A)
  
  R = A - np.diag(np.diag(A))

  Di = np.reciprocal(D)

  MAX_ITERATIONS = 10_0000
  iterations = 0
  while (np.linalg.norm(r) / np.linalg.norm(b)) > TOL and iterations < MAX_ITERATIONS:
    r = x.copy()
    x = (b - (R @ x)) * Di
    r = r - x

    iterations += 1

  #print(np.linalg.norm(r) / np.linalg.norm(b))
  #print("iterations:", iterations)
  return x

# Assignment 2: Gauss-Seidel iteration for $Ax=b$

To implement Gauss-Seidel iteration we need to implement the iteration $L\mathbf{x}^{(k+1)} = b - (A - L)\mathbf{x}^{k}$. This can be done by rewriting the equation in terms of the forward substitusion done to solve the equation. The resulting form is $\mathbf{x}^{(k+1)} = D^{-1}(b - L\mathbf{x}^{(k+1)} - U\mathbf{x}^{(k)})$ with $U$ being the strictly upper triangular part of $A$, $L$ being the strictly lower triangular part of $A$, and $D$ being the diagonal part of $A$.

This can very simply be implemented by keeping one vector $\mathbf{x}$ that is partially updated so that $\mathbf{x}_{j} = \mathbf{x}_i^{(k+1)}$ for all $j < i$ and $\mathbf{x}_{j} = \mathbf{x}_i^{k}$ for all $j > i$. This is implemented as `gauss_seidel_direct(A, b, TOL=1e-16)`

Another way to solve this is to calculate $L^{-1}$ and left predcondition the system with that matrix and do richardson iteration on that system. This is implemented as `gauss_seidel(A, b, TOL=1e-16)`.

In [ ]:
# Assignment 2: Gauss-Seidel iteration for Ax=b
def gauss_seidel(A : np.ndarray, b : np.ndarray, TOL = 1e-16):
  Li = np.linalg.inv(np.tril(A))
  #print("Li", Li)
  #print("b", b)
  M = Li @ A
  c = Li @ b
  #print("M", M)
  #print("c", c)
  return richardson_iteration(M, c, 1, TOL)

def gauss_seidel_direct(A : np.ndarray, b : np.ndarray, TOL = 1e-16):
  n, m = A.shape
  x = np.zeros_like(b)
  r = b.copy()
  D = np.diag(np.diag(A))
  AD = A - D
  iterations = 0
  while np.linalg.norm(r) / np.linalg.norm(b) > TOL:
    r = x.copy()
    for i in range(n):
      sigma = AD[i, :] @ x
      x[i] = (b[i] - sigma) / A[i, i]
    r = r - x
    iterations += 1
  #print(f"Iterations: {iterations}")
  return x


## Assignment 3: Newton's method for scalar nonlinear equation $f(x)=0$

Here I have implemented two variations of Newtons method, one called `newton_proper(f, dfdx, x0)` that takes a function `f(x)` and it's derivative `dfdx(x)` and solves $f(x) = 0$ with an initial guess $x_0$. The second implementation is the secant method `secant_method(f, x0, x1)` which similarly takes in the function `f(x)` as well as two initial guesses $x_0$ and $x_1$.

In [ ]:
# Assignment 3: Newton's methods for scalar nonlinear equation f(x) = 0

from typing import Callable

def newton_proper(f : Callable[[np.double], np.double], dfdx : Callable[[np.double], np.double], x0, TOL = 1e-30):
  x = x0
  while np.abs(f(x)) > TOL:
    x = x - (f(x) / dfdx(x))
  return x

def secant_method(f : Callable[[np.double], np.double], x0, x1, TOL = 1e-30):
  while np.abs(f(x1)) > TOL:
    (x0, x1) = (x1, x1 - f(x1) * ((x1 - x0) / (f(x1) - f(x0))))
  return x1

# **Results**

## Assignment 1: Jacobi iteration for $Ax=b$

To verify that the solution obtained from the jacobi iteration is correct we can check that $|| Ax - b ||$ is equal or very close to zero. We can also compare the accuired solution $x$ to a calculated exact solution $y$, $|| x - y ||$.

In [ ]:
## Assignment 1. Tests

A = np.array([[2, 1, 0], [0, 2, 1], [1, 0, 3]], dtype=np.double)
b = np.array([2, 1, 4], dtype=np.double)
y = np.array([1.0, 0, 1])

solution = jacobi_iteration(A, b)
solution2 = jacobi_iteration_direct(A, b)

display(rf"Jacobi iteration")
display(Math(rf"x = \begin{{bmatrix}} {solution[0]} \\ {solution[1]} \\ {solution[2]} \end{{bmatrix}}"))
display(Math(rf"|| Ax - b || = {np.linalg.norm((A @ solution) - b)}"))
display(Math(rf"|| x - y || = {np.linalg.norm(solution - y)}"))

display(rf"Jacobi iteration direct")
display(Math(rf"x = \begin{{bmatrix}} {solution2[0]} \\ {solution2[1]} \\ {solution2[2]} \end{{bmatrix}}"))
display(Math(rf"|| Ax - b || = {np.linalg.norm((A @ solution2) - b)}"))
display(Math(rf"|| x - y || = {np.linalg.norm(solution2 - y)}"))


'Jacobi iteration'

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

'Jacobi iteration direct'

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

And as we can see the result of both implementations converge. And in this case has zero representable error.

## Assignment 2: Gauss-Seidel iteration for $Ax=b$

To verify this the Gauss-Seidel iteration we do the same tests as for the Jacobi iteration. We compare $Ax$ to $b$ to see how similar they are using $|| Ax - b ||$, and we compare the solution $x$ to a calculated solution $y$ with $|| x - y ||$.

In [ ]:
## Assignment 2. Tests

solution = gauss_seidel(A, b)
solution2 = gauss_seidel_direct(A, b, 1e-17)

display(rf"Gauss-Seidel iteration")
display(Math(rf"x = \begin{{bmatrix}} {solution[0]} \\ {solution[1]} \\ {solution[2]} \end{{bmatrix}}"))
display(Math(rf"|| Ax - b || = {np.linalg.norm((A @ solution) - b)}"))
display(Math(rf"|| x - y || = {np.linalg.norm(solution - y)}"))

display(rf"Gauss-Sidel iteration direct")
display(Math(rf"x = \begin{{bmatrix}} {solution2[0]} \\ {solution2[1]} \\ {solution2[2]} \end{{bmatrix}}"))
display(Math(rf"|| Ax - b || = {np.linalg.norm((A @ solution2) - b)}"))
display(Math(rf"|| x - y || = {np.linalg.norm(solution2 - y)}"))


Li [[ 0.5         0.          0.        ]
 [ 0.          0.5         0.        ]
 [-0.16666667  0.          0.33333333]]
b [2. 1. 4.]
M [[ 1.          0.5         0.        ]
 [ 0.          1.          0.5       ]
 [ 0.         -0.16666667  1.        ]]
c [1.  0.5 1. ]


'Gauss-Seidel iteration'

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

'Gauss-Sidel iteration direct'

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The two implementations converge and the error is not representable.

## Assignment 3: Newton's method for scalar nonlinear equation $f(x)=0$

To verify that the non-linear scalar equation solvers work we can calculate $|f(x)|$ and see if this is indeed zero (or very close to zero). We can also compare this to a calculated answer, in this case the solution to the equation $x^2 - 17 = 0$ is $x = \sqrt{17}$.

In [ ]:
## Assignment 3. Tests

f = lambda x : x * x - 17
dfdx = lambda x : 2 * x

solution = newton_proper(f, dfdx, 2)

solution2 = secant_method(f, -1, 10)

y = np.sqrt(17)

display("Newtons method")
display(Math(rf"|f(x)| = {np.abs(f(solution))}"))
display(Math(rf"| x-y | = {np.abs(solution - y)}"))

display("Secant method")
display(Math(rf"|f(x)| = {np.abs(f(solution2))}"))
display(Math(rf"| x-y | = {np.abs(solution2 - y)}"))

'Newtons method'

<IPython.core.display.Math object>

<IPython.core.display.Math object>

'Secant method'

<IPython.core.display.Math object>

<IPython.core.display.Math object>

As we can see, both newtons method and the secant method converge on the correct result.

# **Discussion**

When implementing these kinds of procedures you often way to put them through their paces by running more tests with more edge cases. For example the iterative methods could check for divergence and report that is some way. They should also be tested on more matrices to guarantee their correctness. This was something I had great trouble getting right with both of the Gauss-Seidel implementation, so there might still exist some issues within these procedures.

For the newtons method implementation I decided not to use something like automatic differentiation or symbolic differentiation as their implementation seemed quite complicated. Insated I opted for implementing one version with explicit drivatives (`newton_proper`) and one version which doesn't require explicit derviatives (`secant_method`).